In [ ]:
import requests
import os
import pandas as pd
import numpy as np

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
url = "https://api.foursquare.com/v3/places/search"
api_key = os.environ['my_key']


headers = {
    "Accept": "application/json",
    "Authorization": api_key  
    
}

In [ ]:
CityByke_locations = pd.read_csv('../data/CityByke.csv')

Latitude = list(CityByke_locations['latitude'])
Longitude = list(CityByke_locations['longitude'])

In [ ]:
name = []
fsq_id =[]
latitude = []
longitude = []
distance = []
address = []
price = []
rating = []

for index in CityByke_locations.index:
    
    params = {
    "fields": "name,fsq_id,categories,rating,location,distance,price,store_id",
    "query": "Restaurants",
    "ll": f"{Latitude[index]},{Longitude[index]}",
    "radius" : 1000,
    # "open_now": "true",
    "sort":"DISTANCE"
    }


    

    response = requests.request("GET", url, params=params, headers=headers)
    response_json = response.json()
    
    for restaurant in response_json['results']:
        
        name.append(restaurant['name'])
        fsq_id.append(restaurant['fsq_id'])
        
        latitude.append(Latitude[index])
        longitude.append(Longitude[index])
        
        distance.append(restaurant['distance'])
        
        try:
            address.append(restaurant['location']['address'])
        except:
            address.append(None)
            
        try:
            price.append(restaurant['price'])
        except:
            price.append(None)
            
        try:
            rating.append(restaurant['rating'])
        except:
            rating.append(None)                      

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#parsed in the for loop above


Put your parsed results into a DataFrame

In [ ]:
restaurant = {"name" : name,
               'fsq_id' : fsq_id,
               # 'category_name' : category_name,
               # 'chains' : chains,
               'latitude' : latitude,
               'longitude' : longitude,
               'distance' : distance,
               'address': address,
               # 'store_id' : store_id,
               # 'locality' : locality,
               'price' : price,
               'rating' : rating
              
              
              }

df_foursquare_restaurants = pd.DataFrame(restaurant)
df_foursquare_restaurants

In [ ]:
# Data Transformation

pointOfInterest = df_foursquare_restaurants.groupby(['latitude', 'longitude']).agg({'fsq_id': 'count', 'distance': np.mean, 'price': np.mean, 'rating':np.mean})
poi_foursquare_restaurants= pointOfInterest.rename(columns= {'fsq_id':'fsq_restaurant_number', 'distance':'Fsq_avg_distance','price': 'Fsq_avg_price', 'rating': 'Fsq_avg_rating' })
poi_foursquare_restaurants.to_csv('../data/foursquare_restaurants.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
yelp_url = "https://api.yelp.com/v3/businesses/search"
api_key = os.environ['yelp_key']


headers = {
    "Accept": "application/json",
    "Authorization": f'Bearer {api_key}'  
    
}


name = []
id = []
category_name = []
latitude = []
longitude = []
distance = []
address = []
price = []
review_count = []
rating = []


for index in CityByke_locations.index:
    
    
    params = {

    "term": "restaurants",
    "latitude": Latitude[index],
    "longitude": Longitude[index],
    "radius" : 1000,
    # "open_now": "true",
    "sort_by":"distance"
    }
    
    
    response = requests.request("GET", yelp_url, params=params, headers=headers)
    response_json = response.json()
    
    
    for business in response_json['businesses']:
        name.append(business['name'])
        id.append(business['id'])
        # category_name.append(business['categories'][0]['title'])
        latitude.append(Latitude[index])
        longitude.append(Longitude[index])
        address.append(business['location']['display_address'])
        distance.append(business['distance'])
        
        try:
            price.append(business['price'])
        except:
            price.append(None)
            
        review_count.append(business['review_count'])
        rating.append(business['rating'])
            


    


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#parsed in the for loop above

Put your parsed results into a DataFrame

In [ ]:
restaurant = {"name" : name,
               'id' : id,
               # 'category_name' : category_name,
               # 'chains' : chains,
               'latitude' : latitude,
               'longitude' : longitude,
               'distance' : distance,
               'address': address,
               # 'store_id' : store_id,
               'price' : price,
               'review_count' : review_count,
               'rating' : rating
              
              
              }   

df_yelp_restaurants = pd.DataFrame(restaurant)

#allign the rating structure with foursquare data
df_yelp_restaurants['adj_rating'] =  df_yelp_restaurants['rating'] *2


# Data Transformation

pointOfInterest = df_yelp_restaurants.groupby(['latitude', 'longitude']).agg({'id': 'count','distance': np.mean , 'review_count': np.mean, 'adj_rating':np.mean})
poi_yelp_restaurants= pointOfInterest.rename(columns= {'id': 'Yelp_restaurant_number', 'distance':'Yelp_avg_distance','review_count' : 'Yelp_avg_review_count', 'adj_rating': 'Yelp_avg_rating' })
poi_yelp_restaurants.to_csv('yelp_restaurants.csv')
poi_yelp_restaurants

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Observing Vancouver, foursquare API seem to have the higher quality of data. I observe it has more attribute. 
For example, foursquare API provide muliple category for each restaurant. so i can observe the different at a glance which restaurant has the most variants of food
It also gives complete information about the price category of each restaurant so i know at a glance how expensive each restaurant is

Get the top 10 restaurants according to their rating

In [ ]:
poi_yelp_restaurants.sort_values('Yelp_avg_rating', ascending=False).head(10)

In [ ]:
poi_foursquare_restaurants.sort_values('fsq_avg_rating', ascending=False).head(10)